In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -r /content/drive/MyDrive/hack_2023_11/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.0 M

In [3]:
! pip install numpy==1.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 12.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


In [1]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import pandas as pd
import os
import numpy as np
import json

# Загрузка данных

In [2]:
seed = 42
np.random.seed(seed)

In [3]:
st_model = SentenceTransformer(
    'cointegrated/rubert-tiny2',
    device='cuda'
)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/6.00 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tinybert-ru-labse-adapter-v2.pt:   0%|          | 0.00/963k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# Подбор кандидатов по базе векторов

In [4]:
automarkup = pd.read_parquet(
    '/content/drive/MyDrive/hack_2023_11/data/automarkup.parquet',
    engine='fastparquet'
)

automarkup = automarkup[~automarkup['query'].isna()]
automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())

In [6]:
automarkup['datetime'] = pd.to_datetime(automarkup['datetime'])
automarkup['month'] = automarkup['datetime'].dt.month

In [8]:
automarkup = automarkup[automarkup['month'].isin([10, 11])].reset_index(drop=True)

In [9]:
videos = pd.read_parquet('/content/drive/MyDrive/hack_2023_11/data/clean_videos_done.parquet')
videos.sample(2)

,video_id,video_title,channel_title,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,clean_title,clean_channel_title
6479006,video_24972264,Как мы снимали в публичном доме!,Секреты Профессионального Развития,2023-09-01 19:00:41+03:00,UGC,Разное,2023-09-27 11:16:28+03:00,как мы снимали в публичном доме,секреты профессионального развития
1521150,video_22470853,Капкейки из цветного теста,Секреты Профессионального Развития,2023-09-01 19:00:41+03:00,UGC,Разное,2023-09-21 14:44:25+03:00,капкейки из цветного теста,секреты профессионального развития


In [10]:
automarkup = automarkup.merge(
    videos[['video_id', 'v_category']].drop_duplicates(subset=['video_id']),
    on='video_id', how='left'
)

In [11]:
del videos

In [12]:
n = 1000
top_n = automarkup['query'].value_counts()[:int(2*n)].index.to_list()
other = list(set(automarkup.groupby(['v_category'])['query'].apply(lambda x: x.drop_duplicates().sample(frac=.3)).tolist()))
queries = list(set(top_n + other))
query2ind = {q: i for i, q in enumerate(queries)}
print(len(queries))

93822


In [13]:
# когда прогоните один раз у вас на диске кандиды уже будут сохранены
# можете поставить значение True, чтобы сэкономить время
use_formed_candidates = True

In [14]:
qembeddings = st_model.encode(
    queries,
    batch_size=1000,
    show_progress_bar=True
)

search_cpu_index = read_index('/content/drive/MyDrive/hack_2023_11/models/candidates_rubert.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

(True, 7000000)

In [15]:
generated_cand_name = '/content/drive/MyDrive/hack_2023_11/models/generated_candidates_rubert.parquet'
if not use_formed_candidates:
    topk = 100
    distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

    generated_cand = {
        'query': [],
        'video_id': []
    }

    for i, q in enumerate(queries):
        vids = faiss_ind[i]
        generated_cand['video_id'] += [ind2videoid[v] for v in vids]
        generated_cand['query'] += [q] * len(vids)

    generated_cand = pd.DataFrame(generated_cand)

    generated_cand.to_parquet(
        generated_cand_name,
        engine='fastparquet'
    )
else:
    generated_cand = pd.read_parquet(
        generated_cand_name,
        engine='fastparquet'
    )